In [17]:
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib
import sys 
import sys  
from scipy import sparse
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
%matplotlib inline  
from lauren_function import *

data_directory = '../data/'

In [18]:
data,feature_names,data_desc = load_data_and_description(data_type='posts_tfidf')

In [19]:
value_counts = data_desc['category'].value_counts()
top_categories = value_counts.index[value_counts > np.percentile(data_desc['category'].value_counts(),97)]
idx = data_desc['category'].isin(top_categories)
data_desc['top category'] = idx.astype(int)

data_desc['top category listed'] = data_desc.ix[data_desc['top category'].values.astype(bool) ,'category']

data_desc['top category listed'] = data_desc['top category listed'].fillna('Other')


In [20]:
data_desc.columns

Index([u'ID', u'author', u'permlink', u'category', u'last_update', u'created',
       u'active', u'last_payout', u'depth', u'children', u'children_rshares2',
       u'net_rshares', u'abs_rshares', u'vote_rshares',
       u'children_abs_rshares', u'cashout_time', u'max_cashout_time',
       u'total_vote_weight', u'reward_weight', u'total_payout_value',
       u'curator_payout_value', u'author_rewards', u'net_votes',
       u'root_comment', u'mode', u'max_accepted_payout',
       u'percent_steem_dollars', u'allow_replies', u'allow_votes',
       u'allow_curation_rewards', u'url', u'root_title',
       u'pending_payout_value', u'total_pending_payout_value', u'active_votes',
       u'replies', u'author_reputation', u'body_language', u'dirty', u'ID.1',
       u'tx_id', u'author.1', u'permlink.1', u'parent_author',
       u'parent_permlink', u'title', u'json_metadata', u'body', u'body urls',
       u'number of body urls', u'number of youtube urls',
       u'number of image urls', u'body tags

In [21]:
data_desc['json_metadata']

0                                                       NaN
1         {"tags":["beauty"],"links":["https://youtu.be/...
2         {"tags":["steemit","steemit-community","steem"...
3         {"tags":["money","cryptotrading","business","c...
4         {"tags":["steem","","introduceyourself","steem...
5         {"tags":["introduceyourself","blockchain","bit...
6         {"tags":["introduceyourself"],"users":["rok-si...
7         {"tags":["photography","steemit","girls","fash...
8         {"tags":["introduceyourself","helloworld"],"im...
9         {"tags":["steemit"],"links":["https://youtu.be...
10                             {"tags":["steem","steemit"]}
11        {"tags":["steemit","steem"],"image":["https://...
12        {"tags":["introduceyourself","steemit","steem"...
13        {"tags":["news","steemit","steem","investing"]...
14        {"tags":["introduceyourself","helloworld"],"im...
15        {"tags":["anarchism","cryptocurrency","mathema...
16        {"tags":["introduceyourself","

In [22]:
data_desc['json_metadata']

0                                                       NaN
1         {"tags":["beauty"],"links":["https://youtu.be/...
2         {"tags":["steemit","steemit-community","steem"...
3         {"tags":["money","cryptotrading","business","c...
4         {"tags":["steem","","introduceyourself","steem...
5         {"tags":["introduceyourself","blockchain","bit...
6         {"tags":["introduceyourself"],"users":["rok-si...
7         {"tags":["photography","steemit","girls","fash...
8         {"tags":["introduceyourself","helloworld"],"im...
9         {"tags":["steemit"],"links":["https://youtu.be...
10                             {"tags":["steem","steemit"]}
11        {"tags":["steemit","steem"],"image":["https://...
12        {"tags":["introduceyourself","steemit","steem"...
13        {"tags":["news","steemit","steem","investing"]...
14        {"tags":["introduceyourself","helloworld"],"im...
15        {"tags":["anarchism","cryptocurrency","mathema...
16        {"tags":["introduceyourself","

In [12]:
data_desc['json_metadata']

0                                                          
1         {"tags":["beauty"],"links":["https://youtu.be/...
2         {"tags":["steemit","steemit-community","steem"...
3         {"tags":["money","cryptotrading","business","c...
4         {"tags":["steem","","introduceyourself","steem...
5         {"tags":["introduceyourself","blockchain","bit...
6         {"tags":["introduceyourself"],"users":["rok-si...
7         {"tags":["photography","steemit","girls","fash...
8         {"tags":["introduceyourself","helloworld"],"im...
9         {"tags":["steemit"],"links":["https://youtu.be...
10                             {"tags":["steem","steemit"]}
11        {"tags":["steemit","steem"],"image":["https://...
12        {"tags":["introduceyourself","steemit","steem"...
13        {"tags":["news","steemit","steem","investing"]...
14        {"tags":["introduceyourself","helloworld"],"im...
15        {"tags":["anarchism","cryptocurrency","mathema...
16        {"tags":["introduceyourself","

In [83]:
def create_tfidf(x):
    try:
        tags = json.loads(x)['tags']
        tags = ','.join(tags)
    except:
        tags = np.nan
        
    return tags

In [84]:
val = data_desc['json_metadata'][597]
print 'val', val, type(val)

val {"image":["https://i.imgur.com/MpxbWe4.jpg"],"links":["http://robotswillstealyourjob.com/read","http://amzn.to/XJoHO4","https://itunes.apple.com/gb/book/robots-will-steal-your-job/id576524472?mt=11","https://sellfy.com/p/sxEG","https://sellfy.com/p/Emz2/"]} <type 'str'>


In [85]:
create_tfidf(val)

nan

In [86]:
data_desc['tags'] = data_desc.apply(lambda x: create_tfidf(x['json_metadata']), axis=1)

In [87]:
data_desc['tags'].fillna(' ', inplace=True)

In [88]:
data_desc['tags']

0                                                          
1                                                    beauty
2                      steemit,steemit-community,steem,chat
3         money,cryptotrading,business,crypto,fintech,po...
4         steem,,introduceyourself,steemit,bitcoin,life,...
5         introduceyourself,blockchain,bitcoin,networkin...
6                                         introduceyourself
7                        photography,steemit,girls,fashion,
8                              introduceyourself,helloworld
9                                                   steemit
10                                            steem,steemit
11                                            steemit,steem
12            introduceyourself,steemit,steem,bitcoin,music
13                             news,steemit,steem,investing
14                             introduceyourself,helloworld
15        anarchism,cryptocurrency,mathematics,steemit,w...
16        introduceyourself,federicopist

In [89]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk import word_tokenize 
from nltk.corpus import stopwords

lmtzr = WordNetLemmatizer()
class WordNetLemmatizer(object):
    def __init__(self):
        self.lmtzr = lmtzr.lemmatize    
    def __call__(self, doc):
        return [self.lmtzr(t) for t in word_tokenize(doc)]
    
tfidfvect = TfidfVectorizer(
    encoding = 'utf-8',
    tokenizer = WordNetLemmatizer(),
    # tokenizer = PorterTokenizer(),
    stop_words = stopwords.words('english'),
    lowercase = True
    
)

posts_tfidf = tfidfvect.fit_transform(data_desc['tags'])

In [90]:
posts_tfidf

<111178x21370 sparse matrix of type '<type 'numpy.float64'>'
	with 463417 stored elements in Compressed Sparse Row format>

In [9]:
import json

topics = []
for idx, row in enumerate(data_desc['json_metadata']):
    if (row != '[]') & pd.notnull(row):
        print json.loads(row)['tags']

[u'beauty']
[u'steemit', u'steemit-community', u'steem', u'chat']
[u'money', u'cryptotrading', u'business', u'crypto', u'fintech', u'politics', u'upvoting', u'disruptive', u'social-media', u'steemenomics', u'financial', u'interest', u'steem', u'great', u'venture-capital', u'steemit-adoption', u'beyondbitcoin', u'crowdfunding', u'cryptocurrencies', u'cryptocurrency', u'investment', u'blockchain', u'cryptonews', u'venturecapital', u'steemit', u'thecryptodrive', u'daily', u'introduction', u'thoughts', u'travel', u'finance', u'market', u'steemitsocool', u'blogging', u'introduceyourself', u'bitcoin', u'introduceemyself', u'economics', u'steemdrive', u'crypto-news', u'competition']
[u'steem', u'', u'introduceyourself', u'steemit', u'bitcoin', u'life', u'steem-ideas']
[u'introduceyourself', u'blockchain', u'bitcoin', u'networking', u'iguana', u'supernet', u'bitcoindark', u'']
[u'introduceyourself']
[u'photography', u'steemit', u'girls', u'fashion', u'']
[u'introduceyourself', u'helloworld']
[

KeyError: 'tags'

In [ ]:
def create_tags(x):
    if (x != '[]') & pd.notnull(x):
        return json.loads(row)['tags']

In [ ]:
data_desc[['number of body tags',
                                   'number of body urls',
                                   'number of image urls',
                                   'number of body mentions',
                                   'number of image urls',
                                   'number of youtube urls',
                                   'language',
                                   'author_reputation_scaled',
                                   'number of steem counts',
           'top category']].isnull().values.any()

In [ ]:
y = data_desc['total_payout_value']

In [ ]:
data_desc['number of steem counts'] = data_desc['number of steem counts'].fillna(0)

In [ ]:
data_desc['number of steem counts'].isnull().values.any()

In [ ]:
columns = ['number of body tags',
           'number of body urls',
           'number of image urls',
           'number of body mentions',
           'number of image urls',
           'number of youtube urls',
           'language',
           'author_reputation_scaled',
           'number of steem counts',
           'top category listed']

In [ ]:
data_desc[columns]

In [ ]:
numberofrecords = data_desc.shape[0]
print 'numberofrecords', numberofrecords

In [ ]:
unique_set_of_authors = set(data_desc['author'])
authors = data_desc[['author']].groupby(['author']).size().nlargest(25).reset_index(name='top25')
print 'aurthors', authors
print 'number of unique authors', len(unique_set_of_authors)

In [ ]:
categories = data_desc[['category']].groupby(['category']).size().nlargest(10).reset_index(name='top10')
print 'categories', categories              

In [ ]:
plt.plot(np.log(y))

In [ ]:
data_desc[['top category']].isnull().values.any()

In [ ]:
data_desc[['author','author_reputation']].sort(['author_reputation'], ascending=False).head()
#test = df.sort(['one'], ascending=[False])#

In [ ]:
data_desc.columns

In [ ]:
value_counts = data_desc['category'].value_counts()
top_categories = value_counts.index[value_counts > np.percentile(data_desc['category'].value_counts(),97)]
idx = data_desc['category'].isin(top_categories)
data_desc['top category'] = idx.astype(int)
data_desc['top category listed'] = data_desc.ix[data_desc['top category'].values.astype(bool) ,'category']
# data_desc['top category listed'] = data_desc['top category listed'].fillna('Other')


In [ ]:
data_desc['top category'].isnull().values.any()

In [ ]:
data_desc['top category listed'].isnull().values.any()

In [ ]:
data_desc['language']

In [ ]:
train_features = data_desc.ix[:,['number of body tags',
                                   'number of body urls',
                                   'number of image urls',
                                   'number of body mentions',
                                   'number of image urls',
                                   'number of youtube urls',
                                   'language',
                                   'author_reputation_scaled',
                                   'number of steem counts',
                                'top category']]

In [ ]:
train_features.isnull().values.any()

In [ ]:
train_features['language']

In [ ]:
data_desc.columns